In [66]:
import os
import sys
import joblib
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS

In [67]:
import warnings
warnings.filterwarnings('ignore')

In [68]:
# If the library is not installed - unlock the field 
!{sys.executable} -m pip install rectools

  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Attempting uninstall: pybind11
    Found existing installation: pybind11 2.10.0
ERROR: Cannot uninstall pybind11 2.10.0, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps pybind11==2.10.0'.


In [69]:
import pandas as pd
import numpy as np
import typing as tp
import requests
import zipfile as zf
import time

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel

from tqdm import tqdm

from lightfm import LightFM
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization
from implicit.als import AlternatingLeastSquares

# LOAD DATA 

In [70]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

In [71]:
req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)


kion dataset download: 100%|██████████| 78.8M/78.8M [04:45<00:00, 276kiB/s]

kion dataset download:   8%|▊         | 6.29M/78.8M [00:00<00:01, 58.3MiB/s]
kion dataset download:  28%|██▊       | 22.0M/78.8M [00:00<00:00, 114MiB/s] 
kion dataset download:  71%|███████   | 55.6M/78.8M [00:00<00:00, 210MiB/s]

In [72]:
files = zf.ZipFile('kion.zip','r')
files.extractall()
files.close()

In [73]:
%%time
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')
interactions = pd.read_csv('data_original/interactions.csv')

# interactions.rename(
#     columns={
#         'track_id': Columns.Item,
#         'last_watch_dt': Columns.Datetime,
#         'total_dur': Columns.Weight
#     },
#     inplace=True)

# interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime])

CPU times: user 3.31 s, sys: 450 ms, total: 3.76 s
Wall time: 3.76 s


# Preprocess

In [74]:
Columns.Datetime = 'last_watch_dt'

In [75]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

In [76]:
interactions.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0



<center><div class="alert alert-block alert-warning" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px;">Добавим трех пользователей, для проверки рекомендаций</b><br>
</div></center>


In [77]:
np.random.seed(42)

In [78]:
# Найдем id для 1ого юзера
interactions[interactions[Columns.User] == 100000] 

,user_id,item_id,last_watch_dt,total_dur,watched_pct


In [79]:
# Найдем id для 2ого юзера
interactions[interactions[Columns.User] == 200000] 

,user_id,item_id,last_watch_dt,total_dur,watched_pct


In [80]:
# Найдем id для 3его юзера
interactions[interactions[Columns.User] == 400000] 

,user_id,item_id,last_watch_dt,total_dur,watched_pct


In [81]:
def random_dates(start, end, n=10):

    ndays = (end - start).days + 1
    return pd.to_timedelta(np.random.randint(0, ndays, n), unit='D') + start

## 1й пользователь, который смотрит только новые фильмы, с 2015 года

In [82]:
item_year = items.query("release_year > 2015")[Columns.Item].unique()
new_users_last_year = pd.DataFrame()
new_users_last_year['item_id'] = item_year[np.random.randint(len(item_year), size=10)]
new_users_last_year[Columns.User] = 100000
new_users_last_year[Columns.Datetime] = random_dates(pd.to_datetime(interactions[Columns.Datetime].min()), 
                                           pd.to_datetime((interactions[Columns.Datetime].max())))
new_users_last_year['total_dur'] = np.random.randint(interactions['total_dur'].quantile(0.1), interactions['total_dur'].quantile(0.9), size=10)
new_users_last_year['watched_pct'] = np.random.randint(100, size=10)

In [83]:
new_users_last_year.head()

,item_id,user_id,last_watch_dt,total_dur,watched_pct
0,4759,100000,2021-05-26,10625,21
1,10754,100000,2021-06-08,13845,88
2,1164,100000,2021-07-07,6991,48
3,12492,100000,2021-06-20,2475,90
4,5073,100000,2021-06-24,5353,58


## 2й пользователь, который смотрит только фэнтези.

In [84]:
item_fi = items[Columns.Item][items['genres'].apply(lambda x: 'фэнтези' in x)].unique()
new_users_fi = pd.DataFrame()
new_users_fi['item_id'] = item_fi[np.random.randint(len(item_fi), size=10)]
new_users_fi[Columns.User] = 200000
new_users_fi[Columns.Datetime] = random_dates(pd.to_datetime(interactions[Columns.Datetime].min()), 
                                           pd.to_datetime((interactions[Columns.Datetime].max())))
new_users_fi['total_dur'] = np.random.randint(interactions['total_dur'].quantile(0.1), interactions['total_dur'].quantile(0.9), size=10)
new_users_fi['watched_pct'] = np.random.randint(100, size=10)

In [85]:
new_users_fi.head()

,item_id,user_id,last_watch_dt,total_dur,watched_pct
0,4711,200000,2021-07-21,8475,70
1,7582,200000,2021-05-02,10275,43
2,7704,200000,2021-07-25,11058,7
3,7582,200000,2021-04-02,7555,46
4,964,200000,2021-05-24,2654,34


## И 3й пользователь, который фанат фильмов США.

In [86]:
item_country = items.query("countries == 'США'")[Columns.Item].unique()
new_users_country = pd.DataFrame()
new_users_country['item_id'] = item_country[np.random.randint(len(item_country), size=10)]
new_users_country[Columns.User] = 400000
new_users_country[Columns.Datetime] = random_dates(pd.to_datetime(interactions[Columns.Datetime].min()), 
                                           pd.to_datetime((interactions[Columns.Datetime].max())))
new_users_country['total_dur'] = np.random.randint(interactions['total_dur'].quantile(0.1), interactions['total_dur'].quantile(0.9), size=10)
new_users_country['watched_pct'] = np.random.randint(100, size=10)

In [87]:
new_users_country.head()

,item_id,user_id,last_watch_dt,total_dur,watched_pct
0,7330,400000,2021-04-25,4929,44
1,11354,400000,2021-08-21,12227,64
2,716,400000,2021-03-26,14594,88
3,11209,400000,2021-06-15,4901,70
4,363,400000,2021-04-29,6373,8


## Добавим наших юзеров в наши данные

In [88]:
print(f'Размер данных до добавления юзеров {interactions.shape[0]}')

interactions = pd.concat([interactions, new_users_last_year, new_users_fi, new_users_country])

print(f'Размер данных после добавления юзеров {interactions.shape[0]}')

Размер данных до добавления юзеров 5476251
Размер данных после добавления юзеров 5476281


#### Все верно! Так как про каждого из юзеров у нас было создано по 10 записей, общий размер данных увеличился ровно на 30 записей


<center><div class="alert alert-block alert-warning" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px;"> Генерация наших матриц между юзером и итемов. item_features, user_fetures</b><br>
</div></center>

In [89]:
#  Переведем даты в тип datetime и найдем max_date
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')
max_date = interactions[Columns.Datetime].max()

# Создадим веса взаимодействий итемов и юзеров. 
## Сделаем свою разбаловку по % просмотра фильма
###  watched_pct => 90% = 10 
###   65 <= watched_pct < 90% = 7
### 30 <= watched_pct < 65% = 5
###  10 <= watched_pct < 30% = 3
### watched_pct < 10% = 1

In [90]:
def create_weight_from_watched_pct(x):
    if x >= 90:
        return 35
    elif x >= 65:
        return 7
    elif x >= 30:
        return 2
    elif x >= 10:
        return 2
    return 1

interactions[Columns.Weight] = interactions['watched_pct'].apply(create_weight_from_watched_pct)

interactions[Columns.Weight].value_counts()


kion dataset download: 100%|██████████| 78.8M/78.8M [00:17<00:00, 210MiB/s]

35    1772275
1     1754939
2     1578594
7      370473
Name: weight, dtype: int64

In [91]:
# Удалим всех итемы с длиной меньше 331с
interactions.drop(interactions.query("total_dur < 331").index, inplace=True)

# Prepare features

## User features

In [92]:
users.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [93]:
users.isnull().sum()

user_id         0
age         14095
income      14776
sex         13831
kids_flg        0
dtype: int64

In [94]:
users.fillna('Unknown', inplace=True)

In [95]:
users.nunique()

user_id     840197
age              7
income           7
sex              3
kids_flg         2
dtype: int64

In [96]:
# Оставим только юзеров, которые есть в date
users = users.loc[users[Columns.User].isin(interactions[Columns.User])].copy()

In [97]:
# Добавим еще одну фичу, которая есть в данных 'kids_flg'

user_features_frames = []
list_features = ["sex", 
                "age", 
                "income", 
                'kids_flg'
                ]
for feature in list_features:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


In [98]:
user_features.query(f"id == 973171")

,id,value,feature
0,973171,М,sex
0,973171,age_25_34,age
0,973171,income_60_90,income
0,973171,1,kids_flg


# Item features|

In [99]:
items.isnull().sum() 
items = items.loc[items[Columns.Item].isin(interactions[Columns.Item])].copy()

In [100]:
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [101]:
items.nunique()

item_id         14109
content_type        2
title           13530
title_orig       9771
release_year      103
genres           2565
countries         668
for_kids            2
age_rating          6
studios            38
directors        7454
actors          11913
description     13878
keywords        13652
dtype: int64

### Genre

In [102]:
# Explode genres to flatten table
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


### Content

In [103]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

In [104]:
content_feature

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


## Добавим из фичей страну фильма и год выпуска фильма

In [105]:
country_feature = items.reindex(columns=[Columns.Item, 'countries'])
country_feature.columns = ["id", "value"]
country_feature["feature"] = "country"
country_feature.head()

,id,value,feature
0,10711,Испания,country
1,2508,США,country
2,10716,Канада,country
3,7868,Великобритания,country
4,16268,СССР,country


In [106]:
print(f'Кол-во разных дат в фиче год выпуска: {items.release_year.nunique()}')

Кол-во разных дат в фиче год выпуска: 103


In [107]:
# так как дат слишком много разобьем фильмы на 3 категории. Фильмые после 2015 года, с 2000 до 2015, и фильмы, которые выпускались ранее
release_year = items.reindex(columns=[Columns.Item, 'release_year'])
release_year.columns = ["id", "value"]
release_year["feature"] = "year"



In [108]:
def year_to_category(year):
    if year >= 2015:
        return 'cat_>2015'
    elif year >= 2000:
        return 'cat_>2000'
    else:
        return 'cat_<2000'
        

In [109]:
release_year = items.reindex(columns=[Columns.Item, 'release_year'])
release_year.columns = ["id", "value"]
release_year["feature"] = "year"
release_year["value"] = release_year["value"].apply(year_to_category)
release_year.head()

,id,value,feature
0,10711,cat_>2000,year
1,2508,cat_>2000,year
2,10716,cat_>2000,year
3,7868,cat_>2015,year
4,16268,cat_<2000,year


In [110]:
item_features = pd.concat((genre_feature, 
                           content_feature,
                           release_year,
                           country_feature     
                          ))

In [111]:
item_features.sample(4)

,id,value,feature
9914,14096,cat_>2000,year
737,427,США,country
3344,2417,приключения,genre
13652,2738,cat_>2000,year


In [112]:
%%time
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=user_features,
    cat_user_features=["sex",
                       "age",
                       "income", 
                      "kids_flg"
                      ],
    item_features_df=item_features,
    cat_item_features=["genre",
                       "content_type", 
                       "year", 
                      "country"
                      ],
)

CPU times: user 1.97 s, sys: 117 ms, total: 2.09 s
Wall time: 2.08 s


# Model

In [127]:
# Вспомним наилучшие параметры для модели после Оптюны 
RANDOM_STATE = 42
NUM_THREADS = 16
K_RECOS = 10

param = {'no_components': 4,
 'loss': 'warp',
 'learning_rate': 0.05152885610815282,
 'item_alpha': 0.001,
 'user_alpha': 0.001,
 'is_fitting_features': True,
 'epochs': 21,
 'model': 'LigthFM'}


In [128]:
model = LightFMWrapperModel(
                LightFM(
                no_components=param["no_components"], 
                loss=param["loss"], 
                random_state=RANDOM_STATE,
                learning_rate=param["learning_rate"],
                user_alpha=param["user_alpha"],
                item_alpha=param["item_alpha"],

            ),
            epochs=param['epochs'],
            num_threads=NUM_THREADS,
        )

In [129]:
model.fit(dataset)

In [130]:
# Проверим рекомендации для наших аватаров и случайного юзера

id_avatars = [100000, 200000, 400000, 13]

#predict_model
recos = model.recommend(
    users=id_avatars,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

In [137]:
recos

,user_id,item_id,score,rank
0,100000,9728,-318.797852,1
1,100000,10440,-319.412688,2
2,100000,13865,-319.538682,3
3,100000,142,-319.710781,4
4,100000,15297,-319.836480,5
5,100000,7793,-319.965604,6
6,100000,7102,-319.990119,7
7,100000,1844,-320.040517,8
8,100000,3734,-320.062627,9
9,100000,4457,-320.135685,10


In [131]:
recos_for_avatar = recos.item_id.values

In [132]:
r1 = set(recos_for_avatar[:10])
r2 = set(recos_for_avatar[10:20])
r3 = set(recos_for_avatar[20:30])

r_id1 = set(recos_for_avatar[30:])

In [ ]:
# Нашим аватарам без фичей юзеров предсказывают итемы - посмотрим пересечения их итемов 
# Обзор и аналатика итемов будет в след ноутбуке
r1 & r2 & r3

In [134]:
# Cлучайного юзера
r_id1

{142, 1844, 2657, 3734, 4151, 4457, 9728, 10440, 13865, 15297}

In [135]:
filename = 'model_LightFM.clf'
joblib.dump(model, filename)

['model_LightFM.clf']

In [136]:
filename = 'data_DATASET.sav'
joblib.dump(dataset, filename)

['data_DATASET.sav']

In [145]:
%%timeit

recos = model.recommend(
    users=[1231],
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

635 ms ± 18.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [165]:
our_users = interactions[Columns.User].unique()

In [201]:
data_reco = None

path = len(our_users) // 5000 

print(f'Кол-во частей, на которые разбиты юзеры - {path}')
print('------- START--------')
for p in tqdm(range(path)):
    # Берем чанки по 5к юзеров
    row =  our_users[p * 5000: (p+1) * 5000]
    
    # Рекомендуем им итемы
    recos = model.recommend(
    users=row,
    dataset=dataset,
    k=10,
    filter_viewed=True,
    )
    
    recos = recos.groupby('user_id')['item_id'].apply(list).reset_index()
    if data_reco is None:
        data_reco = recos
    else:
        data_reco = pd.concat([data_reco, recos])

    #Проверяем длину списков
    if p == 0 or p % 16 == 0:
        print(data_reco.shape)
        
print('------- FINISH--------')
    
    

Кол-во частей, на которые разбиты юзеры - 161
------- START--------


  1%|          | 1/161 [00:05<13:52,  5.20s/it]

(5000, 2)


 11%|█         | 17/161 [01:29<12:25,  5.18s/it]

(85000, 2)


...

(645000, 2)


 90%|█████████ | 145/161 [12:18<01:18,  4.89s/it]

(725000, 2)


100%|██████████| 161/161 [13:38<00:00,  5.08s/it]

(805000, 2)
------- FINISH--------


In [203]:
data_reco.head()

,user_id,item_id
0,208,"[10440, 13865, 9728, 2657, 3734, 4880, 142, 68..."
1,269,"[10440, 9728, 4151, 13865, 3734, 12192, 9996, ..."
2,545,"[3734, 9728, 4151, 4880, 7571, 6809, 11237, 26..."
3,622,"[4151, 4880, 2657, 12192, 9996, 6809, 11237, 4..."
4,897,"[9728, 3734, 4151, 4880, 2657, 142, 6809, 1123..."


In [202]:
data_reco.to_csv('reco_LightFM.csv')